## Into the RLverse: Value iteration algorithm
 Goal: Implementing the algorithms I have learnt to find optimal policies as well as the value functions for the given MDPs (Value Iteration - a special case of Policy Iteration).

 Note: Instructions for running this code are given at the end.

1) Importing the necessary libraries and specifying the engine

In [1]:
import pandas as pd
import numpy as np

2) Defining the value iteration algorithm

In [2]:
# P is the MDP, gamma is the discount factor, theta is the convergence-checking threshold
def value_iteration(P, n_states, n_actions, gamma, theta=1e-10):

    # initialising the state-value function (V-function)
    V = np.zeros(n_states, dtype=np.float64)

    while True: # iterates until the value function converges (which is checked for later)

        # initialising the action-value function (Q-function)
        # it is a 2D array because each state has multiple possible actions, each with it's own Q-value
        # no. of rows = no. of states, no. of columns = no. of actions
        Q = np.zeros((n_states, n_actions), dtype=np.float64)

        # now running the loop to calculate the updated Q-function for every
        # transition of every action of every state:
        for index, (_, initial, action, final, reward, prob) in P.iterrows():
                
            # calculating the action-value function
            Q[initial][action] += prob*(reward + gamma*V[final])

        # if function converges, break
        if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
            iterate=False
            break

        # else, update the state-value function (Policy Improvement)
        # assign each state the value of the maximum returning function in that state
        V = np.max(Q, axis=1)

    # once the value function has converged, calculate the optimal policy pi
    # here it is a function (a lambda) that returns the optimal action for each state
    pi = lambda s: {s:a for s,a in enumerate(np.argmax(Q, axis=1))} [s]

    # return the optimal state-value function and the optimal policy
    return pi, V

3) Assigning the path to the text file containing the MDP, and the file to write output to

In [3]:
input = "C:/Users/eshaa/Desktop/WiDS-RLverse/mdp-10-5.txt"
output = "C:/Users/eshaa/Desktop/WiDS-RLverse/vi_output.txt"

4) Extracting parameters from the MDP

In [4]:
with open(input, 'r') as file:

    for line in file:

        if line.startswith('states'):
            n_states = int(line.split()[1])

        elif line.startswith('actions'):
            n_actions = int(line.split()[1])

        elif line.startswith('gamma'):
            gamma = float(line.split()[1])
            break


5) Reading the MDP data into a dataframe

In [5]:
MDP = pd.read_table(input, sep=' ', names=("_", "initial state", "action taken", "final state", "reward", "transition probability"), skiprows=2, skipfooter=1)

C:\Users\eshaa\AppData\Local\Temp\ipykernel_14016\3956382546.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  MDP = pd.read_table(input, sep=' ', names=("_", "initial state", "action taken", "final state", "reward", "transition probability"), skiprows=2, skipfooter=1)


6) Running the value-iteration algorithm on the MDP to obtain the optimal policy and optimal state-value function

In [6]:
pi, V = value_iteration(MDP, n_states, n_actions, gamma)

7) Writing the output to the output file

In [7]:
with open(output, 'w') as file:

    for state in range(n_states):

        file.write(f'{V[state]:.6f} {pi(state)}\n')

**Instructions for running the above code:**

- Replace addresses in step 5 with the paths to your input and output files.

- Input text file should contain the MDP data in the format:

    states &lt; number of states &gt;\
    actions &lt; number of actions &gt;\
    tran &lt; initial state &gt;  &lt; action taken &gt;  &lt; final state &gt;  &lt; reward &gt;  &lt; transition probability &gt;\
    ...all the other transitions...\
    tran &lt; initial state &gt;  &lt; action taken &gt;  &lt; final state &gt;  &lt; reward &gt;  &lt; transition probability &gt;\
    gamma &lt; discount rate &gt;

- Output text file will contain the output in the format:

    &lt; optimal value function for first state &gt;  &lt; optimal action for first state &gt;\
    ...one entry for each state...\
    &lt; optimal value function for last state &gt;  &lt; optimal action for last state &gt;